In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regex
import itertools # concatenate lists in a list
from itertools import combinations # generating tuples from a list
import networkx as nx # graphs
import matplotlib.pyplot as plt # plotting graph
from collections import Counter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import songs data 
songs1 = pd.read_csv("/content/drive/MyDrive/2021_Magistere/data_recent.csv").set_index('id')
songs2 = pd.read_csv("/content/drive/MyDrive/2021_Magistere/songs.csv").set_index('id')

# Convert artists column from str to list 
def collab_to_list(artists):
    return([x.strip() for x in re.sub('[\[\]\']', ' ', artists).split(',')])

# Keep collaborations of less than 10 artists
def boolean_collab(artist):
    return len(artist) > 1 and len(artist) < 10
    
def the_data(songs):
    # Keep 2000-2020 songs only
    songs = songs[songs['year'] >= 2000]
    # Drop columns and rows we are not interested in
    songs.drop(['mode', 'key'], axis=1, inplace=True)
    songs = songs.drop_duplicates(subset=['name'], keep="last")
    songs['artists'] = songs['artists'].apply(collab_to_list)
    songs = songs[songs['artists'].apply(boolean_collab)]
    return songs

songs = songs1.merge(songs2, how="outer")
songs = the_data(songs)
songs

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,name,popularity,release_date,speechiness,tempo
15701,0.967,2000,0.16000,"[Ludacris, Shawnna]",0.915,275907,0.790,1,0.000000,0.1120,-3.186,What's Your Fantasy (Featuring Shawna),64,2000-01-01,0.2070,140.095
15745,0.892,2000,0.00919,"[Three 6 Mafia, UGK, Project Pat]",0.944,261973,0.839,1,0.000000,0.0921,-4.141,Sippin On Some Syrup (feat. UGK (Underground K...,58,2000-06-13,0.1080,136.019
15815,0.691,2000,0.02490,"[Shyne, Barrington Levy]",0.788,228907,0.594,1,0.000017,0.0534,-4.864,Bad Boyz (feat. Barrington Levy),56,2000,0.3360,83.840
15841,0.731,2000,0.11800,"[Beenie Man, Mya]",0.603,257893,0.782,1,0.000000,0.3630,-4.832,Girls Dem Sugar,57,2000-01-01,0.1620,124.303
15859,0.231,2000,0.95100,"[Emmylou Harris, Alison Krauss, Gillian Welch]",0.653,117800,0.105,0,0.000000,0.1140,-16.511,Didn't Leave Nobody But The Baby,55,2000-12-05,0.0454,52.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278740,0.351,2020,0.02270,"[Trey Songz, Summer Walker]",0.619,194576,0.719,1,0.000000,0.0839,-4.111,Back Home (feat. Summer Walker),69,2020-04-29,0.1570,86.036
278741,0.306,2020,0.17300,"[DripReport, Tyga]",0.875,163800,0.443,1,0.000032,0.0891,-7.461,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012
278742,0.270,2020,0.01670,"[Leon Bridges, Terrace Martin]",0.719,167468,0.385,0,0.031300,0.1110,-10.907,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000
278743,0.153,2020,0.53800,"[Kygo, Oh Wonder]",0.514,180700,0.539,0,0.002330,0.1080,-9.332,How Would I Know,70,2020-05-29,0.1050,123.700


In [5]:
# Import artists data 
artists1 = pd.read_csv("/content/drive/MyDrive/2021_Magistere/data_w_genres.csv")
artists2 = pd.read_csv("/content/drive/MyDrive/2021_Magistere/data_w_genres_recent.csv")
# Get first words in artists genres
def the_genre(genres):
    if len(genres) == 1:
        return genres[0]
    if re.search("band", genres[0]) is not None: # for a boy band : ["boy band", "dance pop", ...]
        return genres[1]
    else:
        return genres[0]
    
def filter_genre(str_genre):
    return (re.search("pop", str_genre) is not None) or (re.search("hip hop", str_genre) is not None) or (re.search("latin", str_genre) is not None)

def the_artists(artists):
    # Drop artists where genre is unknown
    artists = artists[artists['genres']!= '[]']
    artists['new_genre'] = artists['genres'].apply(collab_to_list)
    artists['new_genre'] = artists['new_genre'].apply(the_genre)
    # Keep hip hop, pop or latin artists
    artists = artists[artists["new_genre"].apply(filter_genre)]
    return artists
artists = pd.merge(artists1, artists2, how="outer")
artists = artists.drop_duplicates(subset=['artists'], keep="last")
artists = the_artists(artists)
artists

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres,new_genre
27,-,0.466000,0.446,316307.0,0.546,0.000015,0.1930,-7.022,0.0450,125.961,0.376,55.0,10,0,2,"['alternative pop rock', 'neon pop punk', 'pop...",alternative pop rock
41,11:11,0.174000,0.478,252540.0,0.555,0.000011,0.1030,-7.600,0.1110,128.728,0.139,57.0,5,0,2,"['atl hip hop', 'boy band', 'dance pop', 'gang...",atl hip hop
115,4Minute,0.005890,0.540,192400.0,0.929,0.000004,0.4240,-2.646,0.2780,176.023,0.624,61.0,7,1,2,"['k-pop', 'k-pop girl group']",k-pop
154,A,0.000819,0.259,223267.0,0.899,0.000000,0.5120,-5.948,0.0793,176.686,0.379,48.0,2,1,2,"['dance pop', 'pop', 'post-teen pop']",dance pop
157,A Change of Pace,0.001100,0.528,183893.0,0.946,0.000001,0.0655,-4.874,0.0704,133.074,0.718,43.0,9,1,2,"['neon pop punk', 'pop emo', 'pop punk', 'scre...",neon pop punk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54772,鄭怡,0.536000,0.511,320720.0,0.547,0.000001,0.5840,-10.864,0.0469,74.622,0.372,50.0,11,1,1,"['c-pop', 'classic mandopop', 'taiwan campus f...",c-pop
54777,陳麗斯,0.562000,0.550,195013.0,0.375,0.000004,0.2440,-12.277,0.0332,75.703,0.405,33.0,0,1,2,['classic cantopop'],classic cantopop
54783,鳳飛飛,0.884000,0.358,259387.0,0.208,0.000002,0.1500,-9.524,0.0339,131.261,0.278,43.0,10,1,2,"['c-pop', 'classic mandopop', 'vintage chinese...",c-pop
54786,黃品源,0.541000,0.578,293840.0,0.334,0.000006,0.0675,-11.974,0.0267,135.934,0.243,48.0,9,0,2,"['c-pop', 'classic cantopop', 'classic mandopo...",c-pop


In [6]:
# Computing edges
def get_tuples(collab): # for a list of elements, return the list of possible pairs
    return list(combinations(collab, 2))
edges = list(itertools.chain.from_iterable(songs['artists'].apply(get_tuples))) # append pairs
# Keep edges where both artists are in artists table
edges = [(a1,a2) for (a1,a2) in edges if a1 in list(artists['artists']) and 
         a2 in list(artists['artists'])] 
edges = list(dict(Counter(edges)).keys()) # unique
edges = [(a1, a2) for (a1, a2) in edges if (a2, a1) not in edges]
edges[:10]

[('Fat Joe', 'Ja Rule'),
 ('Fat Joe', 'Ashanti'),
 ('Ja Rule', 'Ashanti'),
 ('Christina Aguilera', 'Mýa'),
 ('Christina Aguilera', 'P!nk'),
 ('Mýa', 'P!nk'),
 ('Diddy', 'Black Rob'),
 ('Jermaine Dupri', 'Ludacris'),
 ('Christina Aguilera', 'Redman'),
 ('Sean Paul', 'Sasha')]

In [7]:
# For each artist, list of artists he/she collaborated with
def list_artists_collab(artist):
    return [a for a in set(list(itertools.chain.from_iterable(
        [t for t in [list(t) for t in edges] if artist in t]))) if a!=artist]
artists['list_artists_collab'] = artists['artists'].apply(list_artists_collab)
# For each artist, number of collaborations with different artists
def nb_artists_collab(liste):
    return len(liste)
artists['nb_artists_collab'] = artists['list_artists_collab'].apply(nb_artists_collab)
# Keep artists who collaborated at least once, in artists database
artists = artists[artists['nb_artists_collab'] > 0]
# For each artist, percentage of collaborations = nb collaborations / nb songs
#artists['percent_collab'] = round((artists['nb_artists_collab']/artists['count'])*100, 2)
def apply_perc(perc):
    if perc > 100:
        return 100
    return perc
#artists['percent_collab'] = artists['percent_collab'].apply(apply_perc)
artists

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres,new_genre,list_artists_collab,nb_artists_collab
382,Adje,0.007390,0.847000,137595.000000,0.573000,1.490000e-02,0.084500,-6.032000,0.253000,99.919000,0.332000,72.000000,0,1,2,"['dutch hip hop', 'dutch trap pop']",dutch hip hop,"[Jhorrmountain, Puri]",2
830,Alida,0.177000,0.570000,208222.000000,0.672000,0.000000e+00,0.076800,-5.890000,0.072300,120.019000,0.448000,86.000000,8,0,2,['norwegian pop'],norwegian pop,[Robin Schulz],1
850,Alkilados,0.195000,0.780000,275946.000000,0.845000,0.000000e+00,0.284000,-4.748000,0.072400,95.981000,0.694000,61.000000,10,1,1,"['colombian pop', 'ecuadorian pop', 'latin', '...",colombian pop,"[Nicky Jam, J Alvarez, El Roockie]",3
1066,Andreas Moss,0.410000,0.511000,242680.000000,0.629000,0.000000e+00,0.162000,-7.755000,0.151000,129.924000,0.044200,67.000000,0,1,2,['scandipop'],scandipop,[NF],1
1568,Arrow Benjamin,0.010300,0.319000,213019.000000,0.846000,4.110000e-05,0.475000,-6.109000,0.079600,139.163000,0.078400,67.000000,6,1,2,['uk pop'],uk pop,[Beyoncé],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54458,will.i.am,0.099131,0.788405,234544.837838,0.731324,2.294676e-05,0.166865,-6.332027,0.120111,119.170514,0.655378,54.027027,1,0,37,"['dance pop', 'pop', 'pop dance', 'pop rap', '...",dance pop,"[Flo Rida, Britney Spears, The Pussycat Dolls,...",8
54478,Ñejo,0.370250,0.787000,323859.750000,0.677000,0.000000e+00,0.127450,-6.904000,0.280250,128.518000,0.682000,63.000000,2,1,4,"['latin', 'latin hip hop', 'perreo', 'reggaeto...",latin,"[Dalmata, Bad Bunny, DJ Nelson]",3
54479,Ñejo & Dalmata,0.161250,0.772000,239474.000000,0.853000,2.920000e-06,0.252500,-5.711000,0.056500,131.466500,0.743000,65.000000,7,1,4,"['latin', 'latin hip hop', 'reggaeton', 'regga...",latin,"[Arcangel, J Alvarez]",2
54480,Ñengo Flow,0.155923,0.749154,358203.153846,0.717000,3.530769e-07,0.260692,-5.635154,0.242454,113.836538,0.575462,66.076923,2,1,13,"['latin', 'latin hip hop', 'reggaeton', 'regga...",latin,"[De La Ghetto, Yandel, Daddy Yankee]",3


In [8]:
# For artist A and artist B, list and number of collabs in common
def boolean_artist(artistA, artistB, collab):
    return artistA in collab and artistB in collab
def boolean_artist_all(artistA, artistB):
    return [boolean_artist(artistA, artistB, collab) for collab in list(songs['artists'])]
def list_collabs(artistA, artistB):
    list_collabs = []
    for song, b in zip(list(songs['name']), boolean_artist_all(artistA, artistB)):
        if b == True:
            list_collabs.append(song)
    return list_collabs
list_collabs_all = [list_collabs(artistA, artistB) for (artistA, artistB) in edges]
nb_collab = [len(c) for c in list_collabs_all]

In [9]:
# Edges dataframe
edges_df = pd.DataFrame({'from' : [a1 for (a1,a2) in edges], 
                         'to' : [a2 for (a1,a2) in edges],
                        'liste_collabs' : list_collabs_all,
                        'nb_collab' : nb_collab})
edges_df = edges_df.sort_values("nb_collab")
edges_df.index = range(len(edges_df.index))
edges_df

,from,to,liste_collabs,nb_collab
0,Fat Joe,Ja Rule,[What's Luv? (feat. Ja-Rule & Ashanti)],1
1,Grimes,Majical Cloudz,[Nightmusic],1
2,Killer Mike,Bun B,"[Big Beast (feat. Bun B, T.I., and Trouble)]",1
3,Killer Mike,Trouble,"[Big Beast (feat. Bun B, T.I., and Trouble)]",1
4,Bun B,Trouble,"[Big Beast (feat. Bun B, T.I., and Trouble)]",1
...,...,...,...,...
2881,Maite Perroni,Christopher von Uckermann,"[Sálvame, Solo Quédate En Silencio, Rebelde, E...",24
2882,RBD,Anahí,"[Sálvame, Solo Quédate En Silencio, Rebelde, E...",24
2883,RBD,Dulce María,"[Sálvame, Solo Quédate En Silencio, Rebelde, E...",24
2884,RBD,Christian Chávez,"[Sálvame, Solo Quédate En Silencio, Rebelde, E...",24


In [10]:
# Nodes dataframe
nodes = list(artists['artists'])
nodes_df = pd.DataFrame({'artist' : nodes,
                        'genre' : list(artists['new_genre']),
                        'genres' : list(artists['genres']),
                        'artists_collab' : list(artists['list_artists_collab']),
                        'degree' : list(artists['nb_artists_collab'])})
nodes_df['sous_genre'] = nodes_df['genre'].copy()

# Definition of genre column
def is_pop(str_genre):
    return re.search("pop", str_genre) is not None
nodes_df.loc[nodes_df["genre"].apply(is_pop), "genre"] = "pop"
def is_hip_hop(str_genre):
    return re.search("hip hop", str_genre) is not None
nodes_df.loc[nodes_df["genre"].apply(is_hip_hop), "genre"] = "hip hop"
def is_latin(str_genre):
    return re.search("latin", str_genre) is not None
nodes_df.loc[nodes_df["genre"].apply(is_latin), "genre"] = "latin"

# Definition of sous_genre column
def is_not_dance_pop(str_sous_genre):
    return re.search("pop", str_sous_genre) is not None and str_sous_genre != "dance pop"
nodes_df.loc[nodes_df["sous_genre"].apply(is_not_dance_pop), "sous_genre"] = "pop"
def is_hip_hop2(str_sous_genre):
    return re.search("hip hop", str_sous_genre) is not None and re.search("atl hip hop", str_sous_genre) is None and re.search("alternative hip hop", str_sous_genre) is None
nodes_df.loc[nodes_df["sous_genre"].apply(is_hip_hop2), "sous_genre"] = "hip hop"
def is_reggaeton(list_genre):
    return "reggaeton" in list_genre and re.search("latin", the_genre(list_genre)) is not None and the_genre(list_genre)!="latin hip hop"
nodes_df.loc[nodes_df["sous_genre"].apply(is_latin), "sous_genre"] = "latin"
nodes_df['genres'] = nodes_df['genres'].apply(collab_to_list)
nodes_df.loc[nodes_df["genres"].apply(is_reggaeton), "sous_genre"] = "reggaeton"

nodes_df = nodes_df.sort_values("genre", ascending=False)
nodes_df.index = range(len(nodes_df.index))
nodes_df.drop(['genres'], axis=1, inplace=True)

nodes_df

,artist,genre,artists_collab,degree,sous_genre
0,KK,pop,"[Shilpa Rao, Shankar-Ehsaan-Loy, Shaan, Shreya...",4,pop
1,Louis Tomlinson,pop,[Bebe Rexha],1,pop
2,"Lisa ""Left Eye"" Lopes",pop,[*NSYNC],1,pop
3,Little Mix,pop,"[Sean Paul, Jason Derulo, Nicki Minaj, Machine...",5,dance pop
4,Lizzo,pop,"[Ariana Grande, Missy Elliott]",2,dance pop
...,...,...,...,...,...
1249,Metro Boomin,hip hop,"[Kash Doll, Young Thug, Future, Drake, Playboi...",8,hip hop
1250,Method Man,hip hop,"[Busta Rhymes, A$AP NAST, Mary J. Blige, Ghost...",7,hip hop
1251,Memphis Bleek,hip hop,"[Missy Elliott, T.I., Freeway]",3,hip hop
1252,Cartel De Santa,hip hop,"[Julieta Venegas, Millonario]",2,hip hop


In [11]:
artists.drop(['mode', 'key', 'new_genre', 'nb_artists_collab', 'list_artists_collab', 'genres'], axis=1, inplace=True)
# Median year column
def artist_in_collab(artist, collab):
    return artist in collab
def artist_in_collab_all(artist):
    return [artist_in_collab(artist, collab) for collab in list(songs['artists'])]
def median_year(artist):
    list_years = []
    for year, b in zip(list(songs['year']), artist_in_collab_all(artist)):
        if b == True:
            list_years.append(year)
    return int(np.median(list_years))
median_year = [median_year(artist) for artist in nodes]

artists['median_year'] = median_year
artists.index = range(len(artists.index))
# Artists dataframe
artists

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,count,median_year
0,Adje,0.007390,0.847000,137595.000000,0.573000,1.490000e-02,0.084500,-6.032000,0.253000,99.919000,0.332000,72.000000,2,2017
1,Alida,0.177000,0.570000,208222.000000,0.672000,0.000000e+00,0.076800,-5.890000,0.072300,120.019000,0.448000,86.000000,2,2020
2,Alkilados,0.195000,0.780000,275946.000000,0.845000,0.000000e+00,0.284000,-4.748000,0.072400,95.981000,0.694000,61.000000,1,2014
3,Andreas Moss,0.410000,0.511000,242680.000000,0.629000,0.000000e+00,0.162000,-7.755000,0.151000,129.924000,0.044200,67.000000,2,2016
4,Arrow Benjamin,0.010300,0.319000,213019.000000,0.846000,4.110000e-05,0.475000,-6.109000,0.079600,139.163000,0.078400,67.000000,2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,will.i.am,0.099131,0.788405,234544.837838,0.731324,2.294676e-05,0.166865,-6.332027,0.120111,119.170514,0.655378,54.027027,37,2010
1250,Ñejo,0.370250,0.787000,323859.750000,0.677000,0.000000e+00,0.127450,-6.904000,0.280250,128.518000,0.682000,63.000000,4,2012
1251,Ñejo & Dalmata,0.161250,0.772000,239474.000000,0.853000,2.920000e-06,0.252500,-5.711000,0.056500,131.466500,0.743000,65.000000,4,2009
1252,Ñengo Flow,0.155923,0.749154,358203.153846,0.717000,3.530769e-07,0.260692,-5.635154,0.242454,113.836538,0.575462,66.076923,13,2014


In [12]:
# Keeping collaborations from artists in the nodes only
def keep_collab(collab):
    return all([c in nodes for c in collab])
songs = songs[songs['artists'].apply(keep_collab)]
# Songs dataframe
songs.index = range(len(songs.index))
songs      

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,name,popularity,release_date,speechiness,tempo
0,0.919,2001,0.02190,"[Fat Joe, Ja Rule, Ashanti]",0.835,267093,0.707,1,0.000022,0.1080,-5.074,What's Luv? (feat. Ja-Rule & Ashanti),73,2001-12-04,0.0598,93.955
1,0.554,2001,0.27300,"[Jennifer Lopez, Ja Rule]",0.708,262133,0.587,1,0.000000,0.0718,-7.930,I'm Real (feat. Ja Rule) - Murder Remix,64,2001-01-23,0.1510,83.460
2,0.679,2001,0.01260,"[Jermaine Dupri, Ludacris]",0.751,200840,0.786,1,0.000000,0.1070,-7.441,Welcome to Atlanta (feat. Ludacris),56,2001-10-30,0.3830,95.989
3,0.436,2002,0.10700,"[Christina Aguilera, Redman]",0.640,298853,0.889,0,0.000000,0.3390,-3.073,Dirrty (feat. Redman),69,2002-07-19,0.3220,99.931
4,0.681,2003,0.00249,"[Beyoncé, JAY-Z]",0.646,236133,0.770,0,0.000000,0.0715,-6.596,Crazy In Love (feat. Jay-Z),76,2003-06-24,0.2260,99.165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.509,2020,0.17500,"[Eminem, Skylar Grey]",0.661,265947,0.748,1,0.000000,0.2540,-4.809,Leaving Heaven (feat. Skylar Grey),71,2020-01-17,0.2060,171.983
1997,0.606,2020,0.33400,"[Chelsea Cutler, Alexander 23]",0.774,148973,0.253,0,0.000001,0.2210,-9.610,Lucky,69,2020-01-17,0.0491,92.122
1998,0.806,2020,0.25500,"[Myke Towers, Yandel]",0.808,131055,0.736,0,0.000273,0.1050,-4.084,Mayor,67,2020-06-08,0.1140,89.983
1999,0.336,2020,0.17300,"[Jeremy Zucker, Chelsea Cutler]",0.358,229205,0.435,0,0.001060,0.0986,-9.198,you were good to me - shallou remix,59,2020-06-12,0.0717,155.908


In [14]:
# Export
nodes_df.to_csv("/content/drive/MyDrive/2021_Magistere/nodes.csv")
edges_df.to_csv("/content/drive/MyDrive/2021_Magistere/edges.csv")
artists.to_csv("/content/drive/MyDrive/2021_Magistere/artists.csv")
songs.to_csv("/content/drive/MyDrive/2021_Magistere/songs.csv")